In [1]:
import os
from pathlib import Path
import pandas as pd
from IPython.display import display, Markdown
import statistics


# ISAdetect
def get_folder_size_ISAdetect(folder_path, min_file_size=0):
    """Calculate the total size of all files in a folder and its subfolders."""
    total_size = 0
    num_files = 0
    for path, dirs, files in os.walk(folder_path):
        for file in files:
            # skip if file is not *.code
            if not file.endswith(".code"):
                continue
            file_path = os.path.join(path, file)
            size = os.path.getsize(file_path)
            if size < min_file_size:
                continue
            num_files += 1
            total_size += size
    return total_size, num_files, total_size / num_files if num_files else 0


# BuildCross
def get_folder_size_BuildCross(folder_path, min_file_size=0):
    """Calculate the total size of all files in a folder and its subfolders."""
    total_size = 0
    num_files = 0
    num_512_files = 0
    num_1024_files = 0
    for path, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(path, file)
            size = os.path.getsize(file_path)
            if size < min_file_size:
                continue
            num_files += 1
            num_512_files += size // 512
            num_1024_files += size // 1024
            total_size += size
    return (
        total_size,
        num_files,
        num_512_files,
        num_1024_files,
        total_size / num_files if num_files else 0,
    )


# CpuRec
def get_file_sizes_CpuRec(folder_path):
    """Get a list of file sizes in a folder and its subfolders."""
    arch_details = {}
    for path, dirs, files in os.walk(folder_path):
        for file in files:
            name = file.split(".")[0]
            file_path = os.path.join(path, file)
            arch_details[name] = {"file_size": os.path.getsize(file_path)}
    return arch_details


features = ["endianness", "instructionwidth_type"]

#### ISAdetect


In [2]:
# SAMPLE CSV
"""
architecture;endianness;wordsize;instructionwidth_type;instructionwidth;comment
alpha;little;64;fixed;32;"none"
amd64;little;64;variable;na;"none"
"""
csv = Path().resolve().parent.parent / "dataset" / "ISAdetect-features.csv"
df = pd.read_csv(csv, delimiter=";")

dataset_folder = (
    Path().resolve().parent.parent / "dataset" / "ISAdetect" / "ISAdetect_full_dataset"
)
total_samples = 0
total_size = 0
arch_details = {}

for folder in dataset_folder.iterdir():
    if folder.is_dir():
        # Get the architecture from the folder name
        architecture = folder.name
        # Get the corresponding row in the DataFrame
        row = df[df["architecture"] == architecture]
        if not row.empty:
            # Extract the features from the DataFrame
            endianness = row.iloc[0]["endianness"]
            instructionwidth_type = row.iloc[0]["instructionwidth_type"]
            # Get the size of the folder
            isa_size, num_files, avg_size = get_folder_size_ISAdetect(folder, 1024)

            arch_details[architecture] = {
                "endianness": endianness,
                "instructionwidth_type": instructionwidth_type,
                "isa_size": isa_size,
                "num_files": num_files,
                "avg_size": avg_size,
            }
            total_samples += num_files
            total_size += isa_size

print("ISAdetect Arch_details: (files smaller than 1024 bytes are ignored)")
print(f"Total Size: {total_size / 1024**2:.2f} MB")
print(f"Average Size per ISA: {total_size / len(arch_details.keys()) / 1024**2:.2f} MB")
# Calculate median size per ISA
median_size = statistics.median(
    [details["isa_size"] for details in arch_details.values()]
)
print(f"Median Size per ISA: {median_size / 1024**2:.2f} MB")

# Prepare markdown table header
md_table = "| Architecture | Endianness | Instruction Width Type | Total Size (MB) | Number of Files |\n"  # Average File Size (MB) |\n"
md_table += "|--------------|------------|-----------------------|-----------------|-----------------|\n"  # -----------------------|\n"

# Fill table rows
for arch in sorted(arch_details.keys()):
    details = arch_details[arch]
    md_table += (
        f"| {arch} | {details['endianness']} | {details['instructionwidth_type']} | "
        f"{details['isa_size']/1024**2:.2f} | {details['num_files']} |\n"  # {details['avg_size']/1024**2:.2f} |\n"
    )

display(Markdown(md_table))

print("Endianness Distribution:")
endianness_distribution = {}
for details in arch_details.values():
    endianness = details["endianness"]
    if endianness not in endianness_distribution:
        endianness_distribution[endianness] = 0
    endianness_distribution[endianness] += details["num_files"]
total = sum(endianness_distribution.values())
md_endianness = "| Endianness | No. samples | Percentage |\n|------------|-------------|------------|\n"
for endianness, count in endianness_distribution.items():
    percent = (count / total) * 100
    md_endianness += f"| {endianness} | {count} | {percent:.2f}% |\n"
display(Markdown(md_endianness))

print("Instruction Width Type Distribution:")
instructionwidth_type_distribution = {}
for details in arch_details.values():
    instructionwidth_type = details["instructionwidth_type"]
    if instructionwidth_type not in instructionwidth_type_distribution:
        instructionwidth_type_distribution[instructionwidth_type] = 0
    instructionwidth_type_distribution[instructionwidth_type] += details["num_files"]
total = sum(instructionwidth_type_distribution.values())
md_instructionwidth = "| Instruction Width Type | No. samples | Percentage |\n|-----------------------|-------------|------------|\n"
for instructionwidth_type, count in instructionwidth_type_distribution.items():
    percent = (count / total) * 100
    md_instructionwidth += f"| {instructionwidth_type} | {count} | {percent:.2f}% |\n"
display(Markdown(md_instructionwidth))
print("==========================")

ISAdetect Arch_details: (files smaller than 1024 bytes are ignored)
Total Size: 15936.77 MB
Average Size per ISA: 692.90 MB
Median Size per ISA: 574.67 MB


| Architecture | Endianness | Instruction Width Type | Total Size (MB) | Number of Files |
|--------------|------------|-----------------------|-----------------|-----------------|
| alpha | little | fixed | 925.77 | 3952 |
| amd64 | little | variable | 564.43 | 4059 |
| arm64 | little | fixed | 418.50 | 3518 |
| armel | little | fixed | 466.91 | 3814 |
| armhf | little | fixed | 331.34 | 3674 |
| hppa | big | fixed | 940.76 | 4830 |
| i386 | little | variable | 519.50 | 4484 |
| ia64 | little | variable | 2044.75 | 4983 |
| m68k | big | variable | 684.06 | 4313 |
| mips | big | fixed | 545.13 | 3547 |
| mips64el | little | fixed | 1117.75 | 4280 |
| mipsel | little | fixed | 545.68 | 3693 |
| powerpc | big | fixed | 547.82 | 3618 |
| powerpcspe | big | fixed | 790.11 | 3922 |
| ppc64 | big | fixed | 771.06 | 2822 |
| ppc64el | little | fixed | 574.67 | 3521 |
| riscv64 | little | fixed | 605.14 | 4285 |
| s390 | big | variable | 360.46 | 5118 |
| s390x | big | variable | 532.86 | 3511 |
| sh4 | little | fixed | 723.25 | 5854 |
| sparc | big | fixed | 362.99 | 4923 |
| sparc64 | big | fixed | 844.07 | 3205 |
| x32 | little | variable | 719.76 | 4059 |


Endianness Distribution:


| Endianness | No. samples | Percentage |
|------------|-------------|------------|
| big | 39809 | 42.36% |
| little | 54176 | 57.64% |


Instruction Width Type Distribution:


| Instruction Width Type | No. samples | Percentage |
|-----------------------|-------------|------------|
| variable | 30527 | 32.48% |
| fixed | 63458 | 67.52% |


#### CpuRec


In [3]:
# SAMPLE CSV
"""
architecture;endianness;wordsize;instructionwidth_type;instructionwidth;comment
alpha;little;64;fixed;32;"none"
amd64;little;64;variable;na;"none"
"""
csv = Path().resolve().parent.parent / "dataset" / "cpu_rec-features.csv"
df = pd.read_csv(csv, delimiter=";")

dataset_folder = (
    Path().resolve().parent.parent / "dataset" / "cpu_rec" / "cpu_rec_corpus"
)
no_samples_endianness = 0
no_samples_instructionwidth_type = 0
total_size = 0
arch_sizes = get_file_sizes_CpuRec(dataset_folder)
arch_details = {}

for arch in arch_sizes.keys():
    # Get the corresponding row in the DataFrame
    row = df[df["architecture"] == arch]
    if not row.empty:
        arch_details[arch] = {}
        # Extract the features from the DataFrame
        endianness = row.iloc[0]["endianness"]
        instructionwidth_type = row.iloc[0]["instructionwidth_type"]
        # Get the size of the folder
        if endianness in ["little", "big"]:
            no_samples_endianness += 1
        else:
            endianness = "unknown"
        if instructionwidth_type in ["fixed", "variable"]:
            no_samples_instructionwidth_type += 1
        else:
            instructionwidth_type = "unknown"
        arch_details[arch]["endianness"] = endianness
        arch_details[arch]["instructionwidth_type"] = instructionwidth_type
        arch_details[arch]["num_files"] = 1
        arch_details[arch]["isa_size"] = arch_sizes[arch]["file_size"]
        total_size += arch_details[arch]["isa_size"]

print("CpuRec Arch_details: ")
print(f"Total Size: {total_size / 1024**2:.2f} MB")

# Prepare markdown table header
md_table = "| Architecture | Endianness | Instruction Width Type | Total Size (kB) |\n"
md_table += "|--------------|------------|-----------------------|-----------------|\n"

# Fill table rows
for arch in sorted(arch_details.keys()):
    details = arch_details[arch]
    md_table += (
        f"| {arch} | {details['endianness']} | {details['instructionwidth_type']} | "
        f"{details['isa_size']/1024:.2f} |\n"
    )

display(Markdown(md_table))

print("Endianness Distribution:")
endianness_distribution = {}
for details in arch_details.values():
    endianness = details["endianness"]
    if endianness == "unknown":
        continue
    if endianness not in endianness_distribution:
        endianness_distribution[endianness] = 0
    endianness_distribution[endianness] += details["num_files"]
total = sum(endianness_distribution.values())
md_endianness = "| Endianness | No. samples | Percentage |\n|------------|-------------|------------|\n"
for endianness, count in endianness_distribution.items():
    percent = (count / total) * 100
    md_endianness += f"| {endianness} | {count} | {percent:.2f}% |\n"
display(Markdown(md_endianness))

print("Instruction Width Type Distribution:")
instructionwidth_type_distribution = {}
for details in arch_details.values():
    instructionwidth_type = details["instructionwidth_type"]
    if instructionwidth_type == "unknown":
        continue
    if instructionwidth_type not in instructionwidth_type_distribution:
        instructionwidth_type_distribution[instructionwidth_type] = 0
    instructionwidth_type_distribution[instructionwidth_type] += details["num_files"]
total = sum(instructionwidth_type_distribution.values())
md_instructionwidth = "| Instruction Width Type | No. samples | Percentage |\n|-----------------------|-------------|------------|\n"
for instructionwidth_type, count in instructionwidth_type_distribution.items():
    percent = (count / total) * 100
    md_instructionwidth += f"| {instructionwidth_type} | {count} | {percent:.2f}% |\n"
display(Markdown(md_instructionwidth))
print("==========================")

CpuRec Arch_details: 
Total Size: 20.98 MB


| Architecture | Endianness | Instruction Width Type | Total Size (kB) |
|--------------|------------|-----------------------|-----------------|
| 6502 | little | variable | 6.57 |
| 68HC08 | big | variable | 18.39 |
| 68HC11 | big | variable | 25.17 |
| 8051 | unknown | variable | 15.76 |
| ARC32eb | big | variable | 46.09 |
| ARC32el | little | variable | 45.88 |
| ARM64 | little | fixed | 345.32 |
| ARMeb | big | fixed | 896.44 |
| ARMel | little | fixed | 329.23 |
| ARMhf | little | fixed | 230.78 |
| ARcompact | little | variable | 118.12 |
| AVR | unknown | variable | 193.40 |
| Alpha | little | fixed | 1065.17 |
| AxisCris | little | variable | 61.11 |
| Blackfin | little | variable | 104.82 |
| CLIPPER | little | variable | 1059.47 |
| Cell-SPU | unknown | unknown | 290.22 |
| CompactRISC | little | variable | 56.58 |
| Cray | unknown | variable | 1120.00 |
| Epiphany | little | variable | 69.03 |
| FR-V | big | fixed | 175.25 |
| FR30 | big | fixed | 141.42 |
| FT32 | little | fixed | 179.18 |
| H8-300 | big | variable | 163.47 |
| H8S | unknown | variable | 81.52 |
| HP-Focus | unknown | variable | 408.00 |
| HP-PA | big | fixed | 1057.03 |
| IA-64 | little | variable | 423.41 |
| IQ2000 | big | fixed | 178.65 |
| M32C | little | variable | 173.75 |
| M32R | big | fixed | 121.89 |
| M68k | big | variable | 728.19 |
| M88k | big | fixed | 351.18 |
| MCore | little | fixed | 101.30 |
| MIPS16 | unknown | fixed | 95.62 |
| MIPSeb | big | fixed | 747.85 |
| MIPSel | little | fixed | 425.16 |
| MMIX | big | fixed | 387.71 |
| MN10300 | little | variable | 114.29 |
| MSP430 | little | variable | 301.51 |
| Mico32 | big | fixed | 163.39 |
| MicroBlaze | big | fixed | 192.71 |
| Moxie | big | variable | 140.64 |
| NDS32 | little | variable | 94.04 |
| NIOS-II | little | fixed | 139.11 |
| PDP-11 | unknown | variable | 124.00 |
| PIC10 | unknown | fixed | 8.89 |
| PIC16 | unknown | fixed | 39.16 |
| PIC18 | unknown | fixed | 45.89 |
| PIC24 | little | fixed | 82.67 |
| PPCeb | big | fixed | 403.82 |
| PPCel | little | fixed | 462.20 |
| RISC-V | little | fixed | 69.24 |
| RL78 | little | variable | 337.46 |
| ROMP | big | variable | 440.00 |
| RX | little | variable | 87.12 |
| S-390 | big | variable | 453.77 |
| SPARC | big | fixed | 1376.51 |
| STM8 | unknown | variable | 15.35 |
| Stormy16 | little | variable | 138.34 |
| SuperH | little | fixed | 876.43 |
| TILEPro | unknown | variable | 112.16 |
| TLCS-90 | unknown | variable | 23.18 |
| TMS320C2x | unknown | variable | 44.94 |
| TMS320C6x | unknown | fixed | 105.53 |
| TriMedia | unknown | unknown | 462.70 |
| V850 | little | variable | 132.65 |
| VAX | little | variable | 318.00 |
| Visium | big | fixed | 274.00 |
| WE32000 | unknown | unknown | 326.32 |
| X86 | little | variable | 396.49 |
| X86-64 | little | variable | 375.41 |
| Xtensa | unknown | variable | 87.56 |
| XtensaEB | big | variable | 66.03 |
| Z80 | little | variable | 20.86 |
| i860 | unknown | fixed | 598.00 |


Endianness Distribution:


| Endianness | No. samples | Percentage |
|------------|-------------|------------|
| big | 23 | 41.07% |
| little | 33 | 58.93% |


Instruction Width Type Distribution:


| Instruction Width Type | No. samples | Percentage |
|-----------------------|-------------|------------|
| variable | 41 | 56.16% |
| fixed | 32 | 43.84% |


##### BuildCross


In [4]:
import statistics

# SAMPLE CSV
"""
architecture;endianness;wordsize;instructionwidth_type;instructionwidth;comment
alpha;little;64;fixed;32;"none"
amd64;little;64;variable;na;"none"
"""
csv = Path().resolve().parent.parent / "dataset" / "buildcross" / "labels.csv"
df = pd.read_csv(csv, delimiter=";")

dataset_folder = Path().resolve().parent.parent / "dataset" / "buildcross" / "text_bin"
total_samples = 0
total_size = 0
arch_details = {}

for folder in dataset_folder.iterdir():
    if folder.is_dir():
        # Get the architecture from the folder name
        architecture = folder.name
        # Get the corresponding row in the DataFrame
        row = df[df["architecture"] == architecture]
        if not row.empty:
            # Extract the features from the DataFrame
            endianness = row.iloc[0]["endianness"]
            instructionwidth_type = row.iloc[0]["instructionwidth_type"]
            # Get the size of the folder
            isa_size, num_files, num_512_files, num_1024_files, avg_size = (
                get_folder_size_BuildCross(folder, 1024)
            )

            arch_details[architecture] = {
                "endianness": endianness,
                "instructionwidth_type": instructionwidth_type,
                "size": isa_size,
                "num_files": num_files,
                "num_1024_files": num_1024_files,
                "num_512_files": num_512_files,
                "avg_size": avg_size,
            }
            total_samples += num_files
            total_size += isa_size

print("BuildCross Arch_details:")
print("(Files smaller than 1024 bytes are ignored)")
print("(No Samples are counted in 1024 bytes, with filesplitting)")
print(f"Total Size: {total_size / 1024**2:.2f} MB")
print(f"Average Size per ISA: {total_size / len(arch_details.keys()) / 1024**2:.2f} MB")
# Calculate median size per ISA
median_size = statistics.median([details["size"] for details in arch_details.values()])
print(f"Median Size per ISA: {median_size / 1024**2:.2f} MB")

# Prepare markdown table header
md_table = "| Architecture | Endianness | Instruction Width Type| Total Size (MB) | Number of Files | No. 1024 sized samples | Average File Size (MB) |\n"
md_table += "|--------------|------------|-----------------------|-----------------|-----------------|------------------------|------------------------|\n"

# Fill table rows
for arch in sorted(arch_details.keys()):
    details = arch_details[arch]
    md_table += (
        f"| {arch} | {details['endianness']} | {details['instructionwidth_type']} | "
        f"{details['size']/1024**2:.2f} | {details['num_files']} | {details['num_1024_files']} | {details['avg_size']/1024**2:.2f} |\n"
    )

display(Markdown(md_table))

print("Endianness Distribution:")
endianness_distribution = {}
endianness_arch_count = {}  # Count of architectures per endianness
for details in arch_details.values():
    endianness = details["endianness"]
    if endianness not in ["little", "big"]:
        continue
    if endianness not in endianness_distribution:
        endianness_distribution[endianness] = 0
        endianness_arch_count[endianness] = 0
    endianness_distribution[endianness] += details["num_1024_files"]
    endianness_arch_count[endianness] += 1

total_samples = sum(endianness_distribution.values())
total_archs = len(arch_details)

md_endianness = "| Endianness | No. 1024 byte samples | Percentage of samples | Architecture count | Percentage of architectures |\n"
md_endianness += "|------------|----------------------|----------------------|-------------------|----------------------------|\n"
for endianness, count in endianness_distribution.items():
    sample_percent = (count / total_samples) * 100
    arch_count = endianness_arch_count[endianness]
    arch_percent = (arch_count / total_archs) * 100
    md_endianness += f"| {endianness} | {count} | {sample_percent:.2f}% | {arch_count} | {arch_percent:.2f}% |\n"
display(Markdown(md_endianness))

print("Instruction Width Type Distribution:")
instructionwidth_type_distribution = {}
instructionwidth_type_arch_count = (
    {}
)  # Count of architectures per instruction width type
for details in arch_details.values():
    instructionwidth_type = details["instructionwidth_type"]
    if instructionwidth_type not in ["fixed", "variable"]:
        continue
    if instructionwidth_type not in instructionwidth_type_distribution:
        instructionwidth_type_distribution[instructionwidth_type] = 0
        instructionwidth_type_arch_count[instructionwidth_type] = 0
    instructionwidth_type_distribution[instructionwidth_type] += details[
        "num_1024_files"
    ]
    instructionwidth_type_arch_count[instructionwidth_type] += 1

total = sum(instructionwidth_type_distribution.values())
md_instructionwidth = "| Instruction Width Type | No. 1024 byte samples | Percentage of samples | Architecture count | Percentage of architectures |\n"
md_instructionwidth += "|-----------------------|----------------------|----------------------|-------------------|----------------------------|\n"
for instructionwidth_type, count in instructionwidth_type_distribution.items():
    percent = (count / total) * 100
    arch_count = instructionwidth_type_arch_count[instructionwidth_type]
    arch_percent = (arch_count / total_archs) * 100
    md_instructionwidth += f"| {instructionwidth_type} | {count} | {percent:.2f}% | {arch_count} | {arch_percent:.2f}% |\n"
display(Markdown(md_instructionwidth))
print("==========================")

BuildCross Arch_details:
(Files smaller than 1024 bytes are ignored)
(No Samples are counted in 1024 bytes, with filesplitting)
Total Size: 119.88 MB
Average Size per ISA: 3.00 MB
Median Size per ISA: 2.51 MB


| Architecture | Endianness | Instruction Width Type| Total Size (MB) | Number of Files | No. 1024 sized samples | Average File Size (MB) |
|--------------|------------|-----------------------|-----------------|-----------------|------------------------|------------------------|
| arc | little | variable | 3.23 | 14 | 3299 | 0.23 |
| arceb | big | variable | 1.70 | 12 | 1731 | 0.14 |
| bfin | little | variable | 2.88 | 14 | 2942 | 0.21 |
| bpf | little | fixed | 0.02 | 1 | 19 | 0.02 |
| c6x | big | fixed | 5.55 | 8 | 5679 | 0.69 |
| cr16 | little | variable | 1.97 | 13 | 2012 | 0.15 |
| cris | little | variable | 3.98 | 14 | 4074 | 0.28 |
| csky | little | variable | 4.15 | 14 | 4247 | 0.30 |
| epiphany | little | variable | 0.46 | 6 | 471 | 0.08 |
| fr30 | big | variable | 2.17 | 7 | 2223 | 0.31 |
| frv | big | fixed | 4.93 | 14 | 5037 | 0.35 |
| ft32 | little | fixed | 0.44 | 9 | 440 | 0.05 |
| h8300 | big | variable | 4.30 | 9 | 4402 | 0.48 |
| iq2000 | big | fixed | 2.41 | 8 | 2466 | 0.30 |
| kvx | little | variable | 4.90 | 14 | 5016 | 0.35 |
| lm32 | big | fixed | 3.32 | 13 | 3396 | 0.26 |
| loongarch64 | little | fixed | 4.71 | 14 | 4818 | 0.34 |
| m32r | big | fixed | 1.96 | 12 | 1997 | 0.16 |
| m68k-elf | big | variable | 1.83 | 12 | 1866 | 0.15 |
| mcore | little | fixed | 1.24 | 7 | 1270 | 0.18 |
| mcoreeb | big | fixed | 1.24 | 7 | 1270 | 0.18 |
| microblaze | big | fixed | 5.74 | 14 | 5867 | 0.41 |
| microblazeel | little | fixed | 5.71 | 14 | 5840 | 0.41 |
| mmix | big | fixed | 4.22 | 13 | 4314 | 0.32 |
| mn10300 | little | variable | 1.70 | 12 | 1732 | 0.14 |
| moxie | big | variable | 2.19 | 12 | 2237 | 0.18 |
| moxieel | little | variable | 2.19 | 12 | 2232 | 0.18 |
| msp430 | little | variable | 0.42 | 5 | 432 | 0.08 |
| nds32 | little | variable | 2.85 | 14 | 2908 | 0.20 |
| nios2 | little | fixed | 4.21 | 14 | 4301 | 0.30 |
| or1k | big | fixed | 5.42 | 14 | 5544 | 0.39 |
| pru | little | fixed | 2.39 | 8 | 2443 | 0.30 |
| rl78 | little | variable | 0.63 | 5 | 643 | 0.13 |
| rx | little | variable | 1.46 | 12 | 1486 | 0.12 |
| tilegx | little | fixed | 11.71 | 14 | 11986 | 0.84 |
| tricore | little | variable | 1.61 | 8 | 1646 | 0.20 |
| v850 | little | variable | 3.53 | 10 | 3609 | 0.35 |
| visium | big | fixed | 3.41 | 12 | 3488 | 0.28 |
| xstormy16 | little | variable | 0.48 | 5 | 490 | 0.10 |
| xtensa | big | variable | 2.61 | 14 | 2669 | 0.19 |


Endianness Distribution:


| Endianness | No. 1024 byte samples | Percentage of samples | Architecture count | Percentage of architectures |
|------------|----------------------|----------------------|-------------------|----------------------------|
| big | 54186 | 44.22% | 16 | 40.00% |
| little | 68356 | 55.78% | 24 | 60.00% |


Instruction Width Type Distribution:


| Instruction Width Type | No. 1024 byte samples | Percentage of samples | Architecture count | Percentage of architectures |
|-----------------------|----------------------|----------------------|-------------------|----------------------------|
| variable | 52367 | 42.73% | 22 | 55.00% |
| fixed | 70175 | 57.27% | 18 | 45.00% |
